# 第三章 刚体运动

> 所有坐标系遵循右手定则，即$\hat{x}\times\hat{y}=\hat{z}$

所有刚体位形都可以通过螺旋运动来实现，从而衍生出6参数位形表示法：**指数坐标（exponential coordinate）**。

- **运动旋量（twist）**或**空间速度（spatial velocity）**：3个角速度和3个线速度
- **力旋量（wrench）**或**空间力（spatial force）**：力和力矩


## 3.1 平面内的刚体运动

**螺旋运动（screw motion）**表示：螺旋轴(screw axis) + 旋转角度

螺旋轴$S=(\omega, v_x, v_y)$, $\omega$为单位角速度, $v_x$和$v_y$为移动速度，旋转角度$\theta$，刚体位移$S\theta$，

角速度与线速度组合称为**运动旋量（twist）**，单位螺旋轴$S=(\omega, v_x, v_y)(\omega=1)$，旋转速度$\dot\theta$，则运动旋量$\mathcal{V} = S\dot\theta$


> 常见的机器人姿态表示方法：轴角法、欧拉角法，RPY角法，凯莱-罗德里格斯参数法，单位四元数法

**查理-莫兹（Chasles-Mozzi）定理**：任一刚体运动都可以通过绕某一螺旋轴旋转一定角度和平移来实现

## 3.2 旋转与角速度

### 旋转矩阵

旋转矩阵中的3个列向量分别对应物体坐标系的3个单位坐标轴，即$(\hat x_b, \hat y_b, \hat z_b)$

3×3的旋转矩阵组成的集合成为特殊正交群（special orthogonal group）$SO(3)$，满足：(1)$R^TR=I$; (2)$det R = 1$

> $SO(n)$群被称为矩阵李群（matrix Lie group），群中元素构成一个微分流形

旋转矩阵特性：

- **封闭性**：$AB$也是该群的一个元素
- **结合律**：$(AB)C=A(BC)$
- **幺元律**：存在单位元素$I$，满足$AI=IA=A$
- **可逆性**：存在可逆单元$A^{-1}$，满足$AA^{-1}=A^{-1}A=I$


旋转矩阵的应用：

- **表示姿态**： $R_{bc}$表示{c}相对于{b}的姿态
- **进行坐标系转换**，通过向量或坐标系来表示：下标消减原则，$R_{ab}R_{bc}=R_{ac}$
- **对向量或坐标系进行旋转变换**：$R_{sc}$既可以看成{c}在{s}中姿态，可以看做{s}到{c}的变换

旋转矩阵左乘和右乘：

- 左乘$R=Rot(\hat{\omega},\theta)$绕固定系中转轴$\hat{\omega}$转动
- 右乘$R=Rot(\hat{\omega},\theta)$绕物体系中转轴$\hat{\omega}$转动

> 个人理解：绕固定系转轴旋转，可以理解成{s}变换到{s'}，{b}在{s'}下的表达，因此$R_{sb'} = R_{s'b} = R_{s's}R_{sb'}$；绕相对系转轴旋转，可以理解成先将{s}变换到{b}，然后将{b}变换到{b''}，因此$R_{sb''} = R_{sb}R_{bb''}$。因此导致了左乘和右乘的区别（旋转轴不变）

定义旋转函数Rot(omega, theta)

In [1]:
import numpy as np # 导入numpy
from numpy import sin, cos

def Rot(omega, theta):
    """General Rotation function

    :param omega: Axis 3x1 vector
    :param theta: Angle scalar
    :return: Rotation Matrix 3x3

    """

    return np.array([[cos(theta)+omega[0]**2*(1-cos(theta)),                   omega[0]*omega[1]*(1-cos(theta))-omega[2]*sin(theta),   omega[0]*omega[2]*(1-cos(theta))+omega[1]*sin(theta)],
                     [omega[0]*omega[1]*(1-cos(theta))+omega[2]*sin(theta),    cos(theta)+omega[1]**2*(1-cos(theta)),                  omega[1]*omega[2]*(1-cos(theta))-omega[0]*sin(theta)],
                     [omega[0]*omega[2]*(1-cos(theta))-omega[1]**2*sin(theta), omega[1]*omega[2]*(1-cos(theta))+omega[0]*sin(theta),   cos(theta)+omega[2]**2*(1-cos(theta))               ]])

测试一下旋转函数返回矩阵是否正确

In [7]:
print(Rot([1, 0, 0], 0.0)) #不旋转单位阵

A = Rot([0, 0, 1], 90.0/180.0*np.pi) #绕z旋转90°
print(A.dot(np.array([1,0,0])))


[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[6.123234e-17 1.000000e+00 0.000000e+00]


> 要旋转速度矢量$v$，只涉及用于表达$v$的坐标系，因此，$\hat\omega$也要在同一坐标系中描述，这个转动后的向量$v'$可写成$v'=Rv$

### 角速度

$\hat\omega$为瞬时单位转动轴，$\dot\theta$表示角速度，其组合表示**角速度（angular velocity）**（这个角速度$\omega$是矢量）
$$
\omega=\hat\omega\dot\theta
$$

在各个轴的速度分量：
$$
\dot{\hat x}=\omega\times\hat x \\
\dot{\hat y}=\omega\times\hat y \\
\dot{\hat z}=\omega\times\hat z
$$

在某一特定时刻$t$，令$\omega_s\in\mathbb{R}^3$表示固定坐标系中的角速度$\omega$，则姿态矩阵$R(t)$的时间变化率$\dot R(t)$表示成矩阵形式：
$$
\dot R(t) = \begin{bmatrix}\omega_s\times r_1 & \omega_s\times r_2 & \omega_s\times r_3 \end{bmatrix} = \omega_s \times R
$$

为了将叉积消去，定义一种新的运算$[\omega_s]R$，其中$[\omega_s]$ 为向量 $\omega_s\in\mathbb{R^3}$ 对应的**3×3反对称（skew-symmetric）矩阵**。
$$
[x] = \begin{bmatrix}
        0    & -x_3 &  x_2 \\
        x_3  &   0  & -x_1 \\
        -x_2 &  x_1 &   0
      \end{bmatrix}
$$

矩阵$[x]$就是与向量$x$相对应的3×3反对称矩阵。满足
$$
[x] = -[x]^T
$$

所有3×3反对称矩阵的集合称为so(3)，是**SO(3)群的李代数（Lie Algebra）**

**旋转矩阵和反对称矩阵特性：**

给定任意$\omega\in\mathbb{R^3}$和$R\in SO(3)$，总能满足
$$
R[\omega]R^T=[R\omega]
$$

基于反对称矩阵定义，姿态矩阵变化率可重写为：
$$
[\omega_s]R=\dot R
[\omega_s] = \dot RR^{-1}
$$

$\omega_s$和$\omega_b$分别表示同一角速度在不同坐标系的表达：
$$
\omega_b=R_{sb}^{-1}\omega_s=R^{-1}\omega_s=R^T\omega_s
$$

令$R(t)$表示物体坐标系相对固定坐标系的姿态矩阵，则旋转体的角速度
$$
\dot RR^{-1}=[\omega_s] \\
R^{-1}\dot R=[\omega_b]
$$

> $\omega_b$并不是相对动坐标系的角速度，$\omega_b$表示的只是相对静坐标系{b}的角速度，**{b}只是与运动刚体随动坐标系瞬时重合**

## 转动的指数坐标表示
